## Data preprocessing 

In order to prepare the dataset for machine learning task a pre processing step is added in this notebook. The main aim of this step is to manage the size of the dataset by filtering relevant value categories, optimizing data types for memory management and removing data that is not relevant for the machine learning task at hand.

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [2]:
import warnings

warnings.filterwarnings("ignore")

Set pandas view options for better readability of large dataframes

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

### Dataset composition

Data is split between two files: accepted and rejected loans. To establish what data is available to the lending company while making the decision for accepting/rejecting loan application we can look at rejected dataset and extract the features which we will look for in the approved dataset to structure our dataframe.

In [4]:
print("Starting pre process of datasets")

Starting pre process of datasets


#### Read the columns of rejected loans

In [7]:
cols = pd.read_csv("rejected_2007_to_2018Q4.csv", nrows=0).columns.tolist()
print(cols)

['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length', 'Policy Code']


Seems we have 9 features that are used to determine the loan acceptance/rejection. 

- Amount of loan requested
- Application date
- Purpose of the loan
- FICO risk score of applicant
- Loan amount to annual income ratio
- Location of potential customer with zip code/state which part of is anonymized
- Length of employment
- Policy code which is calculated and assigned based on FICO score by Lending Club (when application is approved?)

We can load in the approved loan dataset and match columns between accepted and rejected datasets

#### Define the respective columns from accepted dataset and set data types

In [8]:
cols_acc = [
    "loan_amnt",
    "issue_d",
    "title",
    "dti",
    "dti_joint",
    "addr_state",
    "fico_range_low",
    "fico_range_high",
    "emp_length",
    "policy_code",
]

In [9]:
col_types = {
    "loan_amnt": "float32",
    "issue_d": "string",
    "emp_title": "category",
    "emp_length": "category",
    "title": "category",
    "addr_state": "category",
    "dti": "float32",
    "dti_join": "float32",
    "fico_range_low": "float32",
    "fico_range_high": "float32",
    "policy_code": "float32",
}

In [10]:
df_app = pd.read_csv(
    "accepted_2007_to_2018Q4.csv",
    usecols=cols_acc,
    dtype=col_types,
    parse_dates=["issue_d"],
)

#### Dtypes for rejected dataset

In [11]:
colm_types = {
    "Amount Requested": "float32",
    "Application Date": "string",
    "Loan Title": "category",
    "Risk_Score": "float32",
    "Debt-To-Income Ratio": "category",
    "State": "category",
    "Employment Length": "category",
    "Policy Code": "float32",
}

In [12]:
cols_import = [
    "Amount Requested",
    "Application Date",
    "Loan Title",
    "Risk_Score",
    "Debt-To-Income Ratio",
    "State",
    "Employment Length",
    "Policy Code",
]

In [13]:
df_rej = pd.read_csv(
    "rejected_2007_to_2018Q4.csv",
    usecols=cols_import,
    dtype=colm_types,
    parse_dates=["Application Date"],
)

### Clean the employment time

In [12]:
df_app["Risk_Score"] = (df_app["fico_range_low"] + df_app["fico_range_high"]) / 2

### Rename the dataframes and reorder columns

In [13]:
df_rej.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,MD,< 1 year,0.0


In [14]:
df_app = df_app.drop(["fico_range_low", "fico_range_high"], axis=1)

In [15]:
df_rej["Debt-To-Income Ratio"] = df_rej["Debt-To-Income Ratio"].str.replace("%", "")

In [16]:
column_order = [
    "Amount Requested",
    "Employment Length",
    "Application Date",
    "State",
    "Debt-To-Income Ratio",
    "Policy Code",
    "Risk_Score",
]

Reorder columns for concatenation

In [17]:
df_rej = df_rej[column_order]

Add target labels

In [18]:
df_rej["loan"] = 0
df_app["loan"] = 1

In [19]:
column_names = {
    "Amount Requested": "loan_amnt",
    "Employment Length": "emp_length",
    "Application Date": "issue_d",
    "State": "addr_state",
    "Debt-To-Income Ratio": "dti",
    "Policy Code": "policy_code",
    "Risk_Score": "Risk_Score",
}

In [20]:
df_rej = df_rej.rename(columns=column_names)

### Merge both dataframes

In [22]:
print("Merging datasets")

Merging datasets


In [30]:
loan_df = pd.concat([df_app, df_rej], axis=0, ignore_index=True)

In [31]:
loan_df["emp_length"] = loan_df["emp_length"].str.extract("(^\d*)")

### Export the dataframe for EDA and ML

Rename the columns so its nice and tidy for future use

In [35]:
column_names = {
    "loan_amnt": "amount",
    "emp_length": "employment_length",
    "issue_d": "date",
    "title": "purpose",
    "addr_state": "state",
    "dti": "debt_to_income",
    "policy_code": "policy",
    "Risk_Score": "fico",
    "loan": "loan",
}
loan_df = loan_df.rename(columns=column_names)

Shuffle the dataset for easier sampling in the analysis

In [36]:
loan_df = shuffle(loan_df)

Export the dataframe

In [39]:
loan_df.to_csv("all_loans_shuf.csv")

In [40]:
print("Finished")

Finished
